# Transformer(트랜스포머) "Attention is all you need"





Transformer는 2017년 구글이 발표한 논문인 "Attention is all you need" 에서 나온 모델로 기존 seq2seq의 구조인 인코더 디코더를 따르면서도, 논문의 이름처럼 Attention 만으로 구현한 모델입니다.

![title](img/1.png)

## 1. 기존 seq2seq 모델의 한계 및 해결방법

    문제점 :
        1. 입력 시퀀스의 정보가 일부분 손실된다.
        2. 속도가 느리다.
    
    해결방법 : 
        RNN 을 사용하지 말고 인코더 디코더를 모두 "Attention"으로 바꾸자
        트랜스포머는 인코더 디코더를 발전시킨 인코딩입니다.
        가장 큰차이는 RNN을 사용하지 않는다는것 입니다.
    
        트랜스포머를 한단어로 표현한다면 "parallelization"입니다.
        RNN 은 순차적으로 첫값부터 마지막 단어까지 순차적으로 계산해 입력 단어를 인코딩 하는 반면, 
        트랜스포머는 행렬곱으로 한번에 이것을 처리합니다.


***

## 2. 트랜스포머의 구조

![title](img/3.png)

트랜스 포머의 가장큰 특징은 이전에도 말했지만 RNN을 인코더, 디코더에서 제거 했다는것입니다.

디코더에서 번역은 기존 RNN과 동일하게 {START}싸인을 시작으로 {END} 사인까지 번역을 하는것을 볼수 있습니다. 

 이러한 점에서 트랜스포머는 기존의 인코더 디코더의 컨셉을 가지고 있다는것 또한 알 수 있습니다.

***

## 3. Positional Encoding

RNN이 없는 트랜스포머는 어떻게 위치및 순서정보를 활용 할 수 있을까???

Postional Encoding은 인코더 및 디코더의 입력값 마다 상대적인 위치정보를 더해주는 기술입니다.

![title](img/4.png)

포지셔널 인코딩 값들은 어떤 값이길래 위치 정보를 반영해줄 수 있을까???  
트랜스포머는 위치 정보를 가진 값을 만들기 위해 아래 두개의 함수를 사용합니다.

![tilte](img/5.png)

사인함수와 코사인 함수의 값을 임베딩 벡터에 더해주므로서 단어의 순서 정보를 더하여 줍니다.

pos , i, d_model 를 이해하기 위해선
포지셔널 인코딩의 덧셈은 사실 임베딩 벡터가 모여 만들어진 문장 벡터 행렬과 포지셔널 인코딩 행렬의 덧셈 연산을 통해 이루어진다는 점을 이해해야 합니다. 

<!-- 하이퍼파라미터
d_model = 512
트랜스포머의 인코더와 디코더에서의 정해진 입력과 출력의 크기를 의미합니다. 즉, 임베딩 벡터의 크기 또한 dmodel이며, 각 인코더와 디코더가 다음 층의 인코더와 디코더로 값을 보낼 때에도 이 크기를 가집니다. 논문에서는 512의 크기를 가졌습니다.
num_layers = 6
트랜스포머에서 하나의 인코더와 디코더를 층으로 생각하였을 때, 트랜스포머 모델에서 인코더와 디코더가 총 몇 층으로 구성되었는지를 의미합니다. 논문에서는 인코더와 디코더를 각각 총 6개 쌓았습니다.
num_heads = 8
트랜스포머에서는 어텐션을 사용할 때, 1번 하는 것 보다 여러 개로 분할해서 병렬로 어텐션을 수행하고 결과값을 다시 하나로 합치는 방식을 택했습니다. 이때 이 병렬의 개수를 의미합니다.
dff = 2048
트랜스포머 내부에는 피드 포워드 신경망이 존재합니다. 이때 은닉층의 크기를 의미합니다. 피드 포워드 신경망의 입력층과 출력층의 크기는 dmodel의 크기를 가집니다. 
-->


![title](img/6.png)

 - pos : 입력 문장에서의 임베딩 벡터의 위치
 - i : 임베딩 벡터 내의 차원의 인덱스
 - d_model : 트랜스포머의 모든 층의 출력 차원을 의미하는 하이퍼파라미터


위와 같은 포지셔널 인코딩 방법을 사용하면 순서 정보가 보존되는데,  
예를 들어 각 임베딩 벡터에 Postional Encoding값을 더하면 같은 단어라고 하더라도 문장 내의 위치에 따라서 트랜스포머의 입력으로 들어가는 임베딩 벡터의 값이 달라집니다.  
결국 트랜스포머의 입력은 순서 정보가 고려된 임베딩 벡터라고 보면 되겠습니다.

****

## 4. Self Attention (핵심!)

각 단어의 Word Embedding 값에 Positional Encodding 값을 더해준 후에 할것은 Self Attention 연산입니다. 

![title](img/8.png)
<!-- 인코딩에서 이루어지는 Attention 연산을 Self Attention 이라고 합니다. -->
<!-- 기존의 어텐션 함수는 주어진 Query에 대해서 모든 Key 와 유사도를 각각 구합니다 그리고 구해낸 이 유사도를 가중치로 하여 키와 맵핑되어있는 각각의 Value에 반영해 줍니다, 그후 유사도가 반영된 값(Value)를 모두 가중합 하여 리턴합니다 -->



Self-Attention 계산의 가장 첫 단계는 encoder에 입력된 벡터(위 경우에서는 각단어의 embedding 벡터)에게서 부터 각 3개의 벡터를 만들어 내는일입니다.  
각 단어에 대하여 Query, Key, Value 를 생성합니다 이 벡터들은 입력벡터에 대해서 세개의 학습가능한 행렬들(Wq,Wk,Wv)을 각각 곱함으로써 만들어집니다.


Query,Key,Vector 는 각각 벡터의 형태입니다.  
여기서 Query 와 Key값을 곱한것을 Attention Score 라고 합니다.   
이 Attention Score 가 높을수록 연관성이 높고, 낮을 수록 연관성이 낮다고 볼 수 있습니다.  
Attention Score 값을 확률적인 값으로 바꾸기 위해 소프트맥스를 사용하고 그 값에 Value 값을 곱한후 그값들을 모두 더해줍니다.

이렇게 되면 이 최종벡터는 단순히 "I" 가 아닌 문장속에서 전체적인 의미를 가진 벡터가 됩니다.

단어 임베딩은 벡터이므로 입력 문장 전체는 행렬로 표현이 가능합니다.  
Query, Key, Value 값들 또한 벡터이므로 우리는 이를 행렬로 표현이 가능합니다.  
그러므로 모든 단어에 대한 어텐션 연산은 행렬곱으로 한번에 처리가 가능합니다.  
RNN이 처음단어 부터 끝단어까지 순차적으로 계산해야 했던 문제가 이렇게 해결됩니다.

![title](img/9.png)


<!-- 논문에선 softmax를 적용하기 전에 score를 ket값의 차원의 루트를 씌운값으로 나누어 적용시킵니다.
softmax(Score/sqrt(dimention of key)) -->

*** 

## 5. Multi Head Attention

트랜스포머 연구진은 한 번의 어텐션을 하는 것보다 여러번의 어텐션을 병렬로 사용하는 것이 더 효과적이라고 판단하였습니다.  
그래서 d_model의 차원을 num_heads개로 나누어 d_model/num_heads의 차원을 가지는 Q, K, V에 대해서 num_heads개의 병렬 어텐션을 수행합니다.  
논문에서는 하이퍼파라미터인 num_heads의 값을 8로 지정하였고, 8개의 병렬 어텐션이 이루어지게 됩니다.  
다시 말해 위에서 설명한 어텐션이 8개로 병렬로 이루어지게 되는데, 이때 각각의 어텐션 값 행렬을 어텐션 헤드라고 부릅니다.  
이때 가중치 행렬 WQ,WK,WV의 값은 8개의 어텐션 헤드마다 전부 다릅니다.

![MHA](https://wikidocs.net/images/page/31379/transformer17.PNG)

여러개의 Attention Layer을 병렬 처리를 하면 아래와 같은 번역에서 큰 도움을 얻을수 있습니다.  
"The animal didnt cross the street because it was too tire"  

it이 가르키는 것은 ??

![title](img/10.png)

<!--
첫번째 어텐션은 animal 에 포커스를 맞추고 
두번째 어텐션은 street에 포커스를 맞춥니다.
-->

하나의 어텐션으론 이런 모호한 문장을 인코딩 하기 어렵기 때문에 이러한 모호한 정보를 multihead attentiion을 이용해 다양한 관점에서 수집하여 이러한 점을 보완할수 있습니다.

***
## 6. Encoder

![title](img/11.png)



<!-- 
아래 그림은 encoder의 전반적인 구조입니다.
보시다싶이 단어를 워드임베딩 벡터로 전환한 후에 
포지셔널 인코딩을 적용합니다.
그다음 멀티헤드 어테션을 입력합니다.
그결과를 이어 붙여서 또다른 행렬을 만들어 냅니다.
그다음 이 행렬은 어떠한 다른 행렬(wo) 와 곱해져 처음의 최초 워드 임베딩과 동일한 차원을 같는 벡터로 출력이 됩니다.
그 벡터들은 따로따로 또 fc (fooly conneted layer)로 들어가서 입력과 동일한 벡터로 출력이 됩니다. 
-->

처음 단어 워드 임베딩 벡터와 출력임베딩 벡터의 크기가 동일합니다 !!  
이는 인코더 레이어 여러개를 붙여서 사용할 수 있다는 말입니다.


## 7. Decoder

트랜스포머의 Decoder는 어떻게 보면 Encoder와 유사한 구조를 가지고 있습니다.  
동일한 수의 Layer를 가지고 있고,  
Decoder 역시 어텐션 병렬처리를 적극적으로 활용합니다.

![title](img/12.png)

이제 이 Decoder를 통해 어떻게 영어가 한글로 번역되는지 알아보겠습니다.  
<!-- Encoder는 (멀티헤드 어텐션, 피드포워드 레이어, 레지듀얼 커낵션)으로 구성되어 있습니다.  -->
Decoder 는 Encoder와 상당히 비슷하지만 몇가지 차이가 있습니다.  
첫번째로 디코더의 Multi head attention 은 "Masked Mutihead attention" 입니다


<!--Masked 라는 이름이 붙은것은 단순히 Decode Layer에서 지금까지 출력된 값에만 어텐션을 적용하기 위해 붙여진 것입니다.
아직 출력되지 않은 미래의 값에 어텐션을 적용하면 안되기 때문입니다.
Masked Mutihead attention 이부분 이해가 잘 안된다.. -->

그 다음 Mutihead Attention 에서 Encoder와 Decoder의 차이점은 Decoder의 멀티헤드 어텐션은 현재 Decoder의 입력값을 쿼리로 사용하고,  
Encoder의 최종 출력값을 KEY 와 Value 로 사용한다는데 있습니다.

<!-- 쉽게말해 디코더의 현재상태를 쿼리로 인코더에 질문하고 인코더 출력값에서 중요한 값을 키와 벨류로 가지고와 디코더에 다음 단어에 가장 적합한 단어를 예측해서 출력할 수 있게 하는 과정이라고 보면 된다 -->

마지막으로 인코더와 마찬가지로 Feed Foward를 통해서 최종값을 벡터로 출력합니다

***

## 8. Vector 값을 어떻게 실제 단어로 출력할까? (Label Smoothing) 여기서부턴 참고~~~

트랜스포머는 최종단계에도 레이블 스무딩이라는 기술을 사용해 모델의 퍼포먼스를 업그레이드 시킵니다.

보통 soft맥스를 쓰면 레이블을 원핫인코딩으로 변환해 줍니다.  
학습데이터가 깔금한경우 이러한 레이블 스무딩은 필요 없을수 있습니다

하지만 출력 레이블이 노이지한 경우 즉 같은 입력값인데 출력값이 학습데이터에 많을때 레이블 스무딩은 큰 도움이 됩니다.

학습이라는것은 소프트맥스의 출력값과 벡터에 전환된 렐이블의 값을 줄이는 것인데 같은 데이터에서 서로 상이한 정답들이 원핫 인코딩으로 존재한다면 모델 파라미터가 크게 커졌다 작아졌다를 반복하여 학습이 원활하게 되기가 힘듭니다.

예를 들어보자면 thank you 중 한개의 레이블은 고마워고 나머지는 감사합니다 라고 되어있다면 두 학습데이터 모두 잘못된것은 아닙니다 
하지만 원핫 인코딩을 두레이블 적용하면 고마워와 감사합니다는 완전히 다른 상이한 값이 됩니다.
그래서 Thank you에 대한 학습은 레이블이 상이한 이유로 원활히 진행되지 않을수가 있습니다. 이럴경우 레이블 스무딩을 적용하면 고마워와 감사합니다는 원핫인코딩보단 조금은 가까운 벡터가 되고 또한 소프트 맥스 출력값과 레이블 차이 역시 조금은 줄어들어 효율적인 학습을 기대할 수 있습니다

***


## 9. Decoding 과정.gif

![title](img/transformer_decoding_1.gif)

***********

![title](img/transformer_decoding_2.gif)


## 10. Ref

- http://jalammar.github.io/illustrated-transformer/
- https://wikidocs.net/31379
- https://www.youtube.com/watch?v=mxGCEWOxfe8
- https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf


# 실습 

https://colab.research.google.com/drive/1S6eQaYTu3R5kEMusj2u0AsgO8oVp20Nd#scrollTo=E2ixqxzbzrC4

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-beta1
import tensorflow_datasets as tfds
import tensorflow as tf

import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model) :
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    print(angle_rads.shape)


    # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])


    print(sines,cosines)


    #pos_encoding = np.concatenate((sines, cosines), axis=-1)
    # 왜 sines와 cosines 의 dimension이 0이라고 나올까? 
    pos_encoding = tf.concat((sines, cosines), -1)
    pos_encoding = pos_encoding[np.newaxis, ...]    
    return tf.cast(pos_encoding, dtype=tf.float32)


pos_encoding = positional_encoding(50, 512)
print (pos_encoding.shape) # 크기 출력

In [ ]:
aa = np.array([[0,1,2,3,4,5],[1,2,3,4,5,6],[2,3,4,5,6,7],[3,4,5,6,7,8]])
bb = np.array([[0,1,2,3,4,5],[1,2,3,4,5,6],[2,3,4,5,6,7],[3,4,5,6,7,8]])
print(aa.shape)
print(aa[:, 0::2])
print(aa[:, 1::2])
print(np.concatenate((aa,bb),axis=1))

In [ ]:
# 임의의 Query, Key, Value인 Q, K, V 행렬 생성


# Query, Key, Value 가 어떻게 나누는가? 가지고있는정보? 



np.set_printoptions(suppress=True)
temp_k = tf.constant([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[   1,0],
                      [  10,0],
                      [ 100,5],
                      [1000,6]], dtype=tf.float32)  # (4, 2)
temp_q = tf.constant([[0, 10, 0]], dtype=tf.float32)  # (1, 3)

In [ ]:
sess = tf.Session()

def scaled_dot_product_attention(query, key, value, mask): # Q 행렬, K 행렬, V 행렬, 마스크 수행 여부
    matmul_qk = tf.matmul(query, key, transpose_b=True)
    # Q 행렬과 K 행렬을 곱한다. 즉, 어텐션 스코어 행렬을 얻는다.
    # matmul은 두 배열의 행렬곱(matrix product)
    # dot 은 두 배열의 내적곱(dot product)

    dk = tf.cast(tf.shape(key)[-1], tf.float32)
    print(sess.run(dk))
    # dk = d_model/num_heads 논문에선 64


    logits = matmul_qk / tf.math.sqrt(dk)
    # 스케일링.

    print(sess.run(logits))


    attention_weights = tf.nn.softmax(logits, axis=-1)
    # 소프트맥스 함수를 사용하여 어텐션 가중치들. 즉, 어텐션 분포를 얻는다.
    output = tf.matmul(attention_weights, value)
    # 어텐션 분포 행렬과 V 행렬을 곱하여 최종 결과를 얻는다.
    # 위값들을 다 더하면 개념부분에서 보았던 Attention Layer Output벡터값을 얻을수 있다.

    return output, attention_weights
    # 최종 결과와 어텐션 분포 리턴. 어텐션 분포 또한 리턴하는 이유는 아래에서 값을 출력해보며 함수 테스트를 위함.

  # 함수 실행
temp_out, temp_attn = scaled_dot_product_attention(temp_q, temp_k, temp_v, None)
sess = tf.Session()
print(sess.run(temp_attn)) # 어텐션 분포(어텐션 가중치의 나열)
print(sess.run(temp_out)) # 어텐션 값

In [ ]:
temp_q = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=tf.float32)  # (3, 3)
temp_out, temp_attn = scaled_dot_product_attention(temp_q, temp_k, temp_v, None)
sess = tf.Session()
print(sess.run(temp_attn)) # 어텐션 분포(어텐션 가중치의 나열)
print(sess.run(temp_out)) # 어텐션 값

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"): # 정의하기
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads # 8
    self.d_model = d_model # 512

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model) #WQ
    self.key_dense = tf.keras.layers.Dense(units=d_model) #WK
    self.value_dense = tf.keras.layers.Dense(units=d_model) #WV

    self.dense = tf.keras.layers.Dense(units=d_model) #WO

  def split_heads(self, inputs, batch_size): # 아래의 call 함수에서 헤드를 나누기 위해서 호출
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # 1. WQ, WK, WV에 해당하는 밀집층 지나기
    query = self.query_dense(query) # (batch_size, seq_len, d_model) 
    key = self.key_dense(key) # (batch_size, seq_len, d_model)
    value = self.value_dense(value) # (batch_size, seq_len, d_model)

    # 2. 헤드 나누기
    query = self.split_heads(query, batch_size) # (batch_size, num_heads, seq_len, d_model/num_heads)
    key = self.split_heads(key, batch_size) # (batch_size, num_heads, seq_len, d_model/num_heads)
    value = self.split_heads(value, batch_size) # (batch_size, num_heads, seq_len, d_model/num_heads)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    outputs = self.dense(concat_attention)

    return outputs # 최종 결과 리턴